In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/data/processed /movies_clean.csv")
df.head()


,budget,genres,id,keywords,popularity,revenue,runtime,title,vote_average,vote_count,release_year,genre_count,keyword_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",150.437577,2787965087,162.0,Avatar,7.2,11800,2009.0,4,21
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",139.082615,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,2007.0,3,16
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",107.376788,880674609,148.0,Spectre,6.3,4466,2015.0,3,7
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",112.312950,1084939099,165.0,The Dark Knight Rises,7.6,9106,2012.0,4,21
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",43.926995,284139100,132.0,John Carter,6.1,2124,2012.0,3,16


## 2. Log transform ile dağılımı iyileştirme

`budget`, `revenue`, `popularity` ve `vote_count` sütunları sağa çarpık (right-skewed)
dağılıma sahip. Yani çok küçük değerler etrafında yoğunlaşıp, az sayıda ama aşırı büyük
değerler içeriyorlar.

Bu durum:

- Modelin büyük değerlere fazla ağırlık vermesine,
- Ağaç tabanlı olmasa bile bazı modellerin zor öğrenmesine sebep olabilir.

Bu yüzden bu değişkenlere `log1p` (log(1 + x)) uyguluyorum:

- Çok büyük değerleri sıkıştırıyor,
- Sıfır değerlerle sorunsuz çalışıyor,
- Dağılımı daha "dengeli" hale getiriyor.

Yeni oluşturduğum sütunlar:

- `budget_log`
- `revenue_log`
- `popularity_log`
- `vote_count_log`


In [21]:
# Log dönüşümü uyguladığımız sayısal sütunlar
log_cols = ["budget", "revenue", "popularity", "vote_count"]

for col in log_cols:
    df[col + "_log"] = np.log1p(df[col])  # log(1 + x)

df[[c for c in df.columns if c.endswith("_log")]].head()



,budget_log,revenue_log,popularity_log,vote_count_log
0,19.283571,21.748578,5.020174,9.375940
1,19.519293,20.683485,4.942232,8.412055
2,19.316769,20.596199,4.685614,8.404472
3,19.336971,20.804790,4.730153,9.116799
4,19.376192,19.464974,3.805039,7.661527


## 3. Runtime'ı kategorilere bölme (binning)

`runtime` sürekli (continuous) bir değişken. Bunun yanında, filmleri sürelerine göre
kategorilere ayırmak model için daha anlamlı olabilir.

Örneğin:

- **short**: 0–80 dakika
- **medium**: 80–120 dakika
- **long**: 120–160 dakika
- **very_long**: 160+ dakika

Bu işlem:

- Modelin "uzun film mi, kısa film mi?" gibi daha anlamlı ayrımlar öğrenmesini sağlar,
- Özellikle ağaç tabanlı modeller için ek sinyal oluşturur.

Bu kategoriyi `runtime_bin` isimli yeni bir sütun olarak ekliyorum.


In [22]:
bins = [0, 80, 120, 160, 500]
labels = ["short", "medium", "long", "very_long"]

df["runtime_bin"] = pd.cut(
    df["runtime"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

df[["runtime", "runtime_bin"]].head()


,runtime,runtime_bin
0,162.0,very_long
1,169.0,very_long
2,148.0,long
3,165.0,very_long
4,132.0,long


## 4. Film yaşı (movie_age) feature'ı

`release_year` değişkeninden, filmin ne kadar "eski" olduğunu türetiyorum.

Formül:

\[
\text{movie_age} = 2025 - \text{release_year}
\]

Böylece:

- Daha yeni filmler mi, yoksa eski filmler mi daha yüksek `vote_average` almış görebilirim,
- Zaman etkisini (time effect) modele sokmuş oluyorum.

Bu yeni değişkenin adı: `movie_age`.


In [23]:
CURRENT_YEAR = 2025  # referans yıl

df["movie_age"] = CURRENT_YEAR - df["release_year"]
df[["release_year", "movie_age"]].head()


,release_year,movie_age
0,2009.0,16.0
1,2007.0,18.0
2,2015.0,10.0
3,2012.0,13.0
4,2012.0,13.0


## 5. Interaction features (oranlar)

Bazı değişkenleri birbirine oranlayarak daha anlamlı metrikler elde edebilirim. Örneğin:

- **budget_per_minute** = bütçe / runtime  
  - Filmin dakika başına maliyetini gösterir.
  - Daha yüksek bütçe yoğunluğu, prodüksiyon kalitesiyle ilişkili olabilir.

- **popularity_per_vote** = popularity / (vote_count + 1)  
  - Popularity skorunu oy sayısına oranlayarak "göreli popülerlik" yaratıyorum.
  - Az oy almış ama popularity'si yüksek filmler ile, çok oy almış ama görece düşük popularity'si olan filmleri ayırt etmeye yardımcı olur.

Bu tarz interaction feature'lar, modelin ham değişkenlerden çıkaramayacağı ilişkileri öğrenmesine yardımcı olur.


In [24]:
# Dakika başına bütçe
df["budget_per_minute"] = df["budget"] / df["runtime"]

# Oy başına popülerlik (0'a bölmemek için +1)
df["popularity_per_vote"] = df["popularity"] / (df["vote_count"] + 1)

df[["budget_per_minute", "popularity_per_vote"]].head()


,budget_per_minute,popularity_per_vote
0,1.462963e+06,0.012748
1,1.775148e+06,0.030900
2,1.655405e+06,0.024038
3,1.515152e+06,0.012333
4,1.969697e+06,0.020672


## 6. Final feature set ve veriyi kaydetme

Bu aşamada, bir sonraki modelleme adımında kullanmak istediğim feature'ları netleştiriyorum.

Kullanacağım değişkenler:

- Log dönüşümlü sayısallar:
  - `budget_log`, `revenue_log`, `popularity_log`, `vote_count_log`
- Orijinal sayısallar:
  - `runtime`, `genre_count`, `keyword_count`, `movie_age`
- Oran/interaction feature'lar:
  - `budget_per_minute`, `popularity_per_vote`
- Kategorik:
  - `runtime_bin`

Son olarak, feature engineering uygulanmış dataset'i
`movies_fe.csv` adıyla `data/processed` klasörüne kaydediyorum.
Bu dosya, bir sonraki notebook'ta (model optimization) doğrudan kullanılacak.


In [27]:
features = [
    "budget_log",
    "revenue_log",
    "popularity_log",
    "vote_count_log",
    "runtime",
    "runtime_bin",
    "genre_count",
    "keyword_count",
    "movie_age",
    "budget_per_minute",
    "popularity_per_vote",
]

# Şimdilik tüm kolonları saklıyorum; istersen sadece features + target da kaydedebiliriz
df_fe = df.copy()

# Aynı klasöre feature engineered dataset'i yaz
output_path = "/content/data/processed /movies_fe.csv"
df_fe.to_csv(output_path, index=False)
print(f"Feature engineered dataset kaydedildi: {output_path}")


Feature engineered dataset kaydedildi: /content/data/processed /movies_fe.csv
